<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/When_%26_otherwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=01679a7749627415704bbce9a6c64381c701e6f2178fa951bf6d672534456a06
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql.types import (StringType,StructField,
                               StructType,IntegerType,
                               ArrayType,MapType)
from pyspark.sql import SparkSession

from pyspark.sql.functions import col,explode,split
from pyspark.sql.functions import col,lit,when,expr



In [3]:
spark = SparkSession.builder.appName('When_otherwise').getOrCreate()

## When() & Otehrwise()


Its similar to SQL Case When, execute sequence of expression untill it maches the condition and return a value when match.

In [4]:
data = [
    ('Micheal jackson','Artist',51,('black','long_hair')),\
    ('Jacky chan','Actor',45,('asian','short_hair')),\
    ('Mia Khalifa','Actress',22,('arabic','long_hair'))
]

prop_struct = StructType([
    StructField('races',StringType()),\
    StructField('hair_style',StringType())])

schema= StructType([
    StructField('Name',StringType()),\
    StructField('Occupation',StringType()),\
    StructField('Age',IntegerType()),\
    StructField('Properties',prop_struct)])


df = spark.createDataFrame(data,schema)
df.show()
df.printSchema()




+---------------+----------+---+-------------------+
|           Name|Occupation|Age|         Properties|
+---------------+----------+---+-------------------+
|Micheal jackson|    Artist| 51| {black, long_hair}|
|     Jacky chan|     Actor| 45|{asian, short_hair}|
|    Mia Khalifa|   Actress| 22|{arabic, long_hair}|
+---------------+----------+---+-------------------+

root
 |-- Name: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Properties: struct (nullable = true)
 |    |-- races: string (nullable = true)
 |    |-- hair_style: string (nullable = true)



In [19]:
df.select(\
          df.Name, when(condition=df.Name=='Mia Khalifa',value='Dani daniels')
                  .when(condition=df.Name=='Jacky chan', value='Tom cruise')
                  .otherwise(df.Name).alias('Artist_name')
          ).show()

+---------------+---------------+
|           Name|    Artist_name|
+---------------+---------------+
|Micheal jackson|Micheal jackson|
|     Jacky chan|     Tom cruise|
|    Mia Khalifa|   Dani daniels|
+---------------+---------------+



In [21]:
df.withColumn('Changed_artist',when(df['Name'] == 'Mia Khalifa','Dani daniels')
                              .when(df['Age'] == 45,'SRK')
                              .otherwise(df['Name'])

              ).show()

+---------------+----------+---+-------------------+---------------+
|           Name|Occupation|Age|         Properties| Changed_artist|
+---------------+----------+---+-------------------+---------------+
|Micheal jackson|    Artist| 51| {black, long_hair}|Micheal jackson|
|     Jacky chan|     Actor| 45|{asian, short_hair}|            SRK|
|    Mia Khalifa|   Actress| 22|{arabic, long_hair}|   Dani daniels|
+---------------+----------+---+-------------------+---------------+

